In [1]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc
from sklearn.feature_selection import RFE

/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


CPU times: user 626 ms, sys: 2.2 s, total: 2.82 s
Wall time: 7.23 s


In [6]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

s = StandardScaler()
df = s.fit_transform(df)
test = s.transform(test)

print df.shape, test.shape

(69713, 22)
(30037, 21)
CPU times: user 57.1 s, sys: 37.8 s, total: 1min 34s
Wall time: 1min 34s


In [10]:
%%time
np.save('yesterday_train.npy', df)
np.save('yesterday_test.npy', test)

CPU times: user 54 ms, sys: 7.06 s, total: 7.11 s
Wall time: 11.5 s


In [13]:
!ls -sh|grep yes

8.4G yesterday_test.npy
 20G yesterday_train.npy


In [7]:
type(test)

numpy.ndarray

In [10]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G        61G        64G       2.6M       9.3M        75M
-/+ buffers/cache:        61G        64G
Swap:         4.0G       154M       3.8G


### from here

In [2]:
%%time
train = np.load('yesterday_train.npy')
test = np.load('yesterday_test.npy')
y = np.load('y.npy')
test_id = np.load('test_id.npy')

CPU times: user 6 ms, sys: 9.97 s, total: 9.97 s
Wall time: 18.7 s


In [3]:
%%time
model=RandomForestClassifier(random_state=7, n_estimators=600, min_samples_split=5, n_jobs=-1)
rfe = RFE(model, train.shape[1]*4/5, step=0.1, verbose=1)
rfe = rfe.fit(train, y)
# summarize the selection of the attributes
print(rfe.support_)
# print(rfe.ranking_)
index2=rfe.support_

Fitting estimator with 37416 features.
Fitting estimator with 33675 features.
Fitting estimator with 29934 features.
[ True  True  True ...  True  True  True]
CPU times: user 6h 11min 30s, sys: 26.7 s, total: 6h 11min 57s
Wall time: 16min 35s


In [4]:
%%time
print train.shape, test.shape
train, test = train[:,index2], test[:, index2]
print train.shape, test.shape

(69713, 37416) (30037, 37416)
(69713, 29932) (30037, 29932)
CPU times: user 2.95 s, sys: 4.2 s, total: 7.15 s
Wall time: 7.15 s


In [5]:
%%time
m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state  = 7)
print m(X_train, y_train, X_test, y_test)

[(55770, 29932), (55770,), (13943, 29932), (13943,)]
CPU times: user 24.7 s, sys: 8.8 s, total: 33.5 s
Wall time: 33.5 s


In [6]:
%%time
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

CPU times: user 16.2 s, sys: 8.21 s, total: 24.4 s
Wall time: 24.4 s


In [7]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.05,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [8]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=20, 
                verbose_eval=1,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.755173	test-auc:0.74267
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 20 rounds.
[1]	train-auc:0.802796	test-auc:0.779792
[2]	train-auc:0.809161	test-auc:0.784481
[3]	train-auc:0.811009	test-auc:0.787497
[4]	train-auc:0.820971	test-auc:0.790974
[5]	train-auc:0.827992	test-auc:0.793848
[6]	train-auc:0.837947	test-auc:0.801864
[7]	train-auc:0.844503	test-auc:0.803624
[8]	train-auc:0.848659	test-auc:0.805626
[9]	train-auc:0.851399	test-auc:0.803775
[10]	train-auc:0.864605	test-auc:0.812723
[11]	train-auc:0.866927	test-auc:0.812498
[12]	train-auc:0.86926	test-auc:0.813868
[13]	train-auc:0.871811	test-auc:0.813559
[14]	train-auc:0.87279	test-auc:0.813315
[15]	train-auc:0.87431	test-auc:0.813846
[16]	train-auc:0.875769	test-auc:0.814021
[17]	train-auc:0.877613	test-auc:0.812348
[18]	train-auc:0.879122	test-auc:0.813032
[19]	train-auc:0.882652	test-auc:0.813288
[20]	train-auc:0.885482	test-auc:

'!!! used 656.90 s'

In [9]:
# clf.get_fscore()

In [13]:
%%time
dtest = xgb.DMatrix(test)

CPU times: user 14.9 s, sys: 7.02 s, total: 21.9 s
Wall time: 21.9 s


In [14]:
%%time
predb = clf.predict(dtest)


CPU times: user 4.3 s, sys: 1 ms, total: 4.3 s
Wall time: 268 ms


___

In [19]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [21]:
submit.to_csv('xgb1.csv',index=False)

In [22]:
clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba